# Milestone 4: Analysis and Visualisation 

In [1]:
# Preamble - imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sbn 

In [2]:
finance_df = pd.read_csv("df_after_Task_6.csv") # All should be cleaned now, dates parsed, etc. 
# type(finance_df) # confirming it's a pd.df :)  

## Task 1: Current state of loans 
Summarise currently what percentage of the loans are recovered against the investor funding and the total amount funded. 
Visualise the results of the above. 
Visualise what percentage of the total amount would be recovered up to 6 months in the future. 

In [3]:
print(finance_df.dtypes)  # compared with loan_data_dict.md




Unnamed: 0                      int64
id                              int64
member_id                       int64
loan_amount                     int64
term_length_in_months         float64
int_rate                      float64
grade                          object
sub_grade                      object
years_of_employment           float64
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_date                     object
loan_status                    object
payment_plan                   object
purpose                        object
dti                           float64
delinq_2yrs                     int64
earliest_credit_line           object
inq_last_6mths                  int64
open_accounts                   int64
total_accounts                  int64
out_prncp                     float64
total_rec_prncp               float64
total_rec_late_fee            float64
recoveries                    float64
last_payment

**Aside:** Upon trying to find suitable columns to 'summarise currently what percentage of the loans are recovered against hte investor funding and the total amount funded,' I realise that I have also in my 'data cleaning' removed all the potentially appropriate columns for that task, namely: 
- **out_prncp**: Remaining outstanding principal for total amount funded
- **out_prncp_inv**: Remaining outstanding principal for portion of total amount funded by investors
- **total_payment**: Payments received to date for total amount funded

I am shocked by the pedagogical blundering that would lead to such an oversight- students don't know where they're going or what sort of analysis will be asked of them, so they don't know how to properly clean the data without utterly destroying its utility, and the casually wirtten data 'cleaning' prompts encourage a cavalier attitude about maintaining data. 

I guess this means I will be importing from the end of "data cleaning" task 5, and hope that I haven't destroyed anything else I might need later. 

In [4]:
finance_df = pd.read_csv("df_end_of_Task_5.csv")

In [5]:
print(finance_df.dtypes)

Unnamed: 0                      int64
id                              int64
member_id                       int64
loan_amount                     int64
funded_amount                 float64
funded_amount_inv             float64
term_length_in_months         float64
int_rate                      float64
instalment                    float64
grade                          object
sub_grade                      object
years_of_employment           float64
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_date                     object
loan_status                    object
payment_plan                   object
purpose                        object
dti                           float64
delinq_2yrs                     int64
earliest_credit_line           object
inq_last_6mths                  int64
open_accounts                   int64
total_accounts                  int64
out_prncp                     float64
out_prncp_in